# Parallelization

This is a notebook that showcases the effects of parallelization when performing the BdG self-consistency cycle. We work with two physical systems: a single atom and a slab of 100 layers, which correspond to small and large Hamiltonian matrices, respectively. For the config we choose 80x80x80 $k$-points for the single atom and 40x40x1 $k$-points for the slab. In all cases, we will do 5 reps without caring for convergence. We will count two types of times: [1] CPU time (i.e. processing time) and [2] Real time (i.e. wall time). At the end of the day, what we care about is wall time, however processing time is also valuable to understand the effects of parallelization. Note that our study will be split into two broad categories: CPU parallelization and GPU parallelization.

## Imports & Setup

In [1]:
import numpy as np
import scipy as sp

# To have access to tsc module
import sys
import os

# Get the parent directory of the current working directory
gparent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
# Append the parent directory to the PYTHONPATH
sys.path.append(gparent_dir)

# ---------------------------------------------------------------------------------

import config

from tsc.utilities import get_RPTS, get_nndists, get_connections, fermi, get_KPTS
from tsc.basis_atoms import single_atom, extract_atom_vectors, slab
from tsc.hamiltonian import hopping_elements, get_exponentials
from tsc.hamiltonian import prep_N_hamiltonian, get_N_hamiltonian

# Import stuff for vectorized full-k Hamiltonian 
from tsc.hamiltonian import prep_N_hamiltonian_vectorized, get_N_hamiltonian_vectorized

# Import stuff for parallelization
from joblib import Parallel, delayed, cpu_count
import functools

# Import cupy for GPU operations
import cupy as cp

import time

# Get config variables
globals().update({k: v for k, v in vars(config.Config).items() if not k.startswith("__")})

In [2]:
def do_stuff(atoms, a, Nkx, Nky, Nkz):
    
    # Construct the lattice sites matrix
    RPTS = get_RPTS(a_1, a_2, a, NCELLS)

    N_k = Nkx*Nky*Nkz

    # Extract the vectors from the called function
    TPTS, atom_types, E_0, U, n_bar, B, Λ = extract_atom_vectors(atoms)
    
    # Write down the number of basis atoms
    N_b: int = TPTS.shape[0]
    
    num_neighbs = get_nndists(RPTS, TPTS, R_max)
    
    # Get the maximum number of neighbours
    max_neighb = num_neighbs.max()
    
    # Get the atom_IJ and Rvec_ij matrices
    atom_IJ, Rvec_ij = get_connections(max_neighb, RPTS, TPTS, R_max)
    
    # Get the number of different atom types based on the basis_atoms configuration
    # Also re-encode the atom_types as 0, 1, 2, etc.
    unique_atoms, atom_types = np.unique(atom_types, return_inverse=True)
    N_unique: int = unique_atoms.shape[0]
    
    # Construct a completely symmetrical case
    t_0 = np.ones((N_unique, N_unique))
    
    # Get the hopping elements
    t = hopping_elements(atom_IJ, num_neighbs, Rvec_ij, atom_types, t_0, R_0)

    # Get a k-mesh with resolution that depends on N_x, N_y, N_z
    KPTS = get_KPTS(a_1, a_2, a, Nkx, Nky, Nkz)

    # Get the exponentials
    fourier = get_exponentials(Rvec_ij, KPTS)

    μ = μ_0
    n = np.ones((N_b))*n_0

    return N_b, E_0, μ, U, n, n_bar, B, atom_IJ, num_neighbs, fourier, t, KPTS, N_k

In [3]:
# Setup config stuff
a_atom = np.array([0.0,0.0,1.0])
a_slab = np.array([0.0,0.0,200.0])

Nkx_a, Nky_a, Nkz_a = 80, 80, 80
Nkx_s, Nky_s, Nkz_s = 40, 40, 1

## CPU Operations

For the CPU Operations we contrust the Hamiltonian matrix separately per $k$-point, using a numba jitted function.

### 1.1.1: Single Atom - No Optimization

In [4]:
atoms = single_atom()

N_b, E_0, μ, U, n, n_bar, B, atom_IJ, num_neighbs, fourier, t, KPTS, N_k = do_stuff(atoms, a_atom, Nkx_a, Nky_a, Nkz_a)

In [5]:
for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    for k in range(N_k):
        # Get a deepcopy of H_prep so that we don't have to re-generate it
        H_copied = np.copy(H_prep)
        # Get a new H(k) for every k
        H = get_N_hamiltonian(k, H_copied, atom_IJ, num_neighbs, fourier, t)

        # Diagonalize H(k)
        w, v = sp.linalg.eigh(H, driver="ev")

        # Store results
        E_vals[k, :] = w
        E_vecs[k, :, :] = v
    end1 = time.process_time()
    end2 = time.time()
    
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------

Processing time = 1.5781.	 Real time = 5.3000
Processing time = 2.2344.	 Real time = 5.1540
Processing time = 1.6094.	 Real time = 5.1560
Processing time = 1.1719.	 Real time = 5.3059
Processing time = 1.2500.	 Real time = 5.1521


### 1.1.2: Single Atom - Parallel, No Batching

In [6]:
def k_loop(k, H_prep, atom_IJ, num_neighbs, fourier, t, N_b):
    H_copied = np.copy(H_prep)
    H = get_N_hamiltonian(k, H_copied, atom_IJ, num_neighbs, fourier, t)
    w, v = sp.linalg.eigh(H, driver="ev")
    return w, v, k

In [7]:
for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k) for k in range(N_k))

    for w, v, k in results:
        E_vals[k, :] = w
        E_vecs[k, :, :] = v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------

Processing time = 2.1406.	 Real time = 9.6579
Processing time = 0.4688.	 Real time = 7.7289
Processing time = 1.1406.	 Real time = 7.1671
Processing time = 1.5469.	 Real time = 7.5790
Processing time = 1.2969.	 Real time = 7.4680


### 1.1.3: Single Atom - Parallel, Batched

In [8]:
def k_batch_loop(start, end, H_prep, atom_IJ, num_neighbs, fourier, t, N_b):
    
    batch_w = np.zeros((end - start, 2*N_b))
    batch_v = np.zeros((end - start, 2*N_b, 2*N_b), dtype=np.complex128)
    for i, k in enumerate(range(start, end)):
        H_copied = np.copy(H_prep)
        H = get_N_hamiltonian(k, H_copied, atom_IJ, num_neighbs, fourier, t)
        w, v = sp.linalg.eigh(H, driver="ev")
        batch_w[i, :] = w
        batch_v[i, :, :] = v
        
    return batch_w, batch_v, start, end

In [9]:
# Batch the parallelism
n_cores = cpu_count()
batch_size = N_k // n_cores

for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_batch_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k, min(k+batch_size, N_k)) for k in range(0, N_k, batch_size))

    for batch_w, batch_v, start, end in results:
        E_vals[start:end, :] = batch_w
        E_vecs[start:end, :, :] = batch_v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 0.1250.	 Real time = 1.2390
Processing time = 0.0156.	 Real time = 1.1528
Processing time = 0.0156.	 Real time = 1.1040
Processing time = 0.0156.	 Real time = 1.1051
Processing time = 0.0000.	 Real time = 1.1330


### 1.2.1: Slab - No Optimization

In [18]:
atoms = slab(slab_length=100)

N_b, E_0, μ, U, n, n_bar, B, atom_IJ, num_neighbs, fourier, t, KPTS, N_k = do_stuff(atoms, a_slab, Nkx_s, Nky_s, Nkz_s)

In [19]:
for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    for k in range(N_k):
        # Get a deepcopy of H_prep so that we don't have to re-generate it
        H_copied = np.copy(H_prep)
        # Get a new H(k) for every k
        H = get_N_hamiltonian(k, H_copied, atom_IJ, num_neighbs, fourier, t)

        # Diagonalize H(k)
        w, v = sp.linalg.eigh(H, driver="ev")

        # Store results
        E_vals[k, :] = w
        E_vecs[k, :, :] = v
    end1 = time.process_time()
    end2 = time.time()
    
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------

Processing time = 2.8594.	 Real time = 27.9318
Processing time = 4.5156.	 Real time = 29.5529
Processing time = 8.3594.	 Real time = 28.8752
Processing time = 3.4062.	 Real time = 27.8049
Processing time = 5.6562.	 Real time = 27.6250


### 1.2.2: Slab - Parallel, No Batching

In [20]:
for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k) for k in range(N_k))

    for w, v, k in results:
        E_vals[k, :] = w
        E_vecs[k, :, :] = v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------

Processing time = 0.2188.	 Real time = 2.7961
Processing time = 0.1250.	 Real time = 3.1583
Processing time = 0.2812.	 Real time = 2.5751
Processing time = 0.4844.	 Real time = 2.8556
Processing time = 0.4375.	 Real time = 2.5173


### 1.2.3: Slab - Parallel, Batched

In [21]:
for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_batch_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k, min(k+batch_size, N_k)) for k in range(0, N_k, batch_size))

    for batch_w, batch_v, start, end in results:
        E_vals[start:end, :] = batch_w
        E_vecs[start:end, :, :] = batch_v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 0.2656.	 Real time = 9.9057
Processing time = 0.2656.	 Real time = 9.8019
Processing time = 0.4531.	 Real time = 9.2620
Processing time = 0.2344.	 Real time = 8.8081
Processing time = 0.2969.	 Real time = 8.6928


Let's see what happens if we experiment with the batch size manually.

In [22]:
batch_size = 20

for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_batch_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k, min(k+batch_size, N_k)) for k in range(0, N_k, batch_size))

    for batch_w, batch_v, start, end in results:
        E_vals[start:end, :] = batch_w
        E_vecs[start:end, :, :] = batch_v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 0.4531.	 Real time = 1.9330
Processing time = 0.3750.	 Real time = 1.7702
Processing time = 0.1719.	 Real time = 1.7288
Processing time = 0.3438.	 Real time = 1.7608
Processing time = 0.3438.	 Real time = 1.8724


It becomes evident that choosing smaller batch sizes allows us to achieve better than non-batched/non-parallel times.

Note that if we set the batch size equal to 1, we get a situation which is equivalent to this of the non-batched parallelization.

In [23]:
batch_size = 1

for rep in range(5):
    
    # Create placeholder arrays for the eigenvalues and eigenvectors
    E_vals, E_vecs = np.zeros((N_k, 2*N_b)), np.zeros((N_k, 2*N_b, 2*N_b), dtype=np.complex128)

    # prepare the Hamiltonian
    H_prep = prep_N_hamiltonian(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3)

    start1 = time.process_time()
    start2 = time.time()
    
    partial_compute_for_k = functools.partial(k_batch_loop, H_prep=H_prep, atom_IJ=atom_IJ, num_neighbs=num_neighbs, fourier=fourier, t=t, N_b=N_b)
    results = Parallel(n_jobs=-1)(delayed(partial_compute_for_k)(k, min(k+batch_size, N_k)) for k in range(0, N_k, batch_size))

    for batch_w, batch_v, start, end in results:
        E_vals[start:end, :] = batch_w
        E_vecs[start:end, :, :] = batch_v
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 0.1719.	 Real time = 2.9119
Processing time = 0.2812.	 Real time = 3.4490
Processing time = 0.2031.	 Real time = 2.8449
Processing time = 0.1875.	 Real time = 2.7302
Processing time = 0.3906.	 Real time = 2.5571


## GPU Operations

For the GPU Operations we construct the Hamiltonian matrix directly for all $k$-points, pass it to the GPU and then diagonalize it in parallel.

### 2.1: Single Atom

In [4]:
atoms = single_atom()

N_b, E_0, μ, U, n, n_bar, B, atom_IJ, num_neighbs, fourier, t, KPTS, N_k = do_stuff(atoms, a_atom, Nkx_a, Nky_a, Nkz_a)

In [9]:
for rep in range(5):

    # Get the full-k prepared Hamiltonian
    H = prep_N_hamiltonian_vectorized(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3, N_k)

    start1 = time.process_time()
    start2 = time.time()

    # Get the full-k actual Hamiltonian
    H = get_N_hamiltonian_vectorized(H, atom_IJ, num_neighbs, fourier, t)

    H_gpu = cp.asarray(H)

    w, v = cp.linalg.eigh(H_gpu)
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # Otherwise we may get memory errors
    cp.get_default_memory_pool().free_all_blocks()

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 0.1406.	 Real time = 0.2740
Processing time = 0.0000.	 Real time = 0.2653
Processing time = 0.0469.	 Real time = 0.2574
Processing time = 0.0000.	 Real time = 0.2589
Processing time = 0.1562.	 Real time = 0.2742
Processing time = 0.1562.	 Real time = 0.2581
Processing time = 0.0469.	 Real time = 0.2529
Processing time = 0.0312.	 Real time = 0.2563
Processing time = 0.0156.	 Real time = 0.2671
Processing time = 0.0625.	 Real time = 0.2574
Processing time = 0.0312.	 Real time = 0.2589
Processing time = 0.2031.	 Real time = 0.2587
Processing time = 0.0000.	 Real time = 0.2634
Processing time = 0.0469.	 Real time = 0.2670
Processing time = 0.1562.	 Real time = 0.2591


### 2.2: Slab

In [11]:
atoms = slab(slab_length=100)

N_b, E_0, μ, U, n, n_bar, B, atom_IJ, num_neighbs, fourier, t, KPTS, N_k = do_stuff(atoms, a_slab, Nkx_s, Nky_s, Nkz_s)

In [12]:
for rep in range(5):

    # Get the full-k prepared Hamiltonian
    H = prep_N_hamiltonian_vectorized(E_0, μ, U, n, n_bar, B, s_0, s_1, s_2, s_3, N_k)

    start1 = time.process_time()
    start2 = time.time()

    # Get the full-k actual Hamiltonian
    H = get_N_hamiltonian_vectorized(H, atom_IJ, num_neighbs, fourier, t)

    H_gpu = cp.asarray(H)

    w, v = cp.linalg.eigh(H_gpu)
    
    end1 = time.process_time()
    end2 = time.time()
    print(f"Processing time = {end1-start1:.4f}.\t Real time = {end2-start2:.4f}")

    # Otherwise we may get memory errors
    cp.get_default_memory_pool().free_all_blocks()

    # ---------------------------------------------------------
    # We do not care about the rest stuff
    # ---------------------------------------------------------


Processing time = 11.0469.	 Real time = 24.3263
Processing time = 8.2188.	 Real time = 24.9634
Processing time = 11.4219.	 Real time = 24.1042
Processing time = 10.7500.	 Real time = 24.5035
Processing time = 10.5469.	 Real time = 24.5470


## Conclusions

Obviously, parallelization does wonders when it comes to parallelizable tasks - this is no newsflash. By properly batching the data that are distributed we can achieve better than "default" times. When it comes to GPU vs CPU parallelization, it is evident that for relatively small matrices (e.g. a single atom, or just a few atoms), the GPU is ideal. However, when the matrix size increases (e.g. a slab), the diagonalization becomes more complex and requires the power of a CPU to run fast.